## Data-QA for event.mediawiki_edit_attempt
### Create new test data and track events created by my user id

**Issues**
- Discusstion Tools 
    - events were logged even with Ad Blocker on
    - all events logged successfully
    
- VE
    - init, ready, loaded, abort were logged successfully 
    - Some events were missing save success, intent and attempt events
- Wikitext editor
    - none of the events were logged


In [2]:
from wmfdata import hive, spark

### Round 1    
#### Date : Nov 8, 2022 
 
User: Maya testwiki  
User_id: 47306  



In [15]:
query1=spark.run('''
SELECT 
  *
  FROM event.mediawiki_edit_attempt
  WHERE performer.id=47306
  ORDER BY dt
''')

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [32]:
query1.tail()

,_schema,agent,custom_data,dt,http,mediawiki,meta,name,page,performer,user_agent_map,is_wmf_domain,normalized_host,datacenter,year,month,day,hour
55,None,"(None, mediawiki_js, desktop_browser)","{'integration': ('string', 'discussiontools'),...",2022-11-08T22:55:55.550Z,"(None, None, None, {'user-agent': 'Mozilla/5.0...","(None, None, None, None, None, 1.40.0-wmf.8, N...","(test.wikipedia.org, 2022-11-08T22:56:00.991Z,...",eas.dt.abort,"(Test5555 Name, None, 147141, None, 1, None, 5...","(None, 5, None, None, 47306, None, True, None,...","{'os_family': 'Mac OS X', 'browser_major': '10...",True,"(wikipedia, test, [], org, wikipedia)",codfw,2022,11,8,22
56,None,"(None, mediawiki_js, desktop_browser)","{'integration': ('string', 'page'), 'editing_s...",2022-11-08T23:00:06.155Z,"(None, None, None, {'user-agent': 'Mozilla/5.0...","(None, None, None, None, None, 1.40.0-wmf.8, N...","(test.wikipedia.org, 2022-11-08T23:00:11.288Z,...",eas.ve.init,"(Test5555 Name, None, 54295, None, 0, None, 0,...","(None, 9, None, None, 47306, None, True, None,...","{'os_family': 'Mac OS X', 'browser_major': '10...",True,"(wikipedia, test, [], org, wikipedia)",codfw,2022,11,8,23
57,None,"(None, mediawiki_js, desktop_browser)","{'integration': ('string', 'page'), 'editing_s...",2022-11-08T23:00:07.035Z,"(None, None, None, {'user-agent': 'Mozilla/5.0...","(None, None, None, None, None, 1.40.0-wmf.8, N...","(test.wikipedia.org, 2022-11-08T23:00:11.290Z,...",eas.ve.ready,"(Test5555 Name, None, 54295, None, 0, None, 0,...","(None, 9, None, None, 47306, None, True, None,...","{'os_family': 'Mac OS X', 'browser_major': '10...",True,"(wikipedia, test, [], org, wikipedia)",codfw,2022,11,8,23
58,None,"(None, mediawiki_js, desktop_browser)","{'integration': ('string', 'page'), 'editing_s...",2022-11-08T23:00:07.036Z,"(None, None, None, {'user-agent': 'Mozilla/5.0...","(None, None, None, None, None, 1.40.0-wmf.8, N...","(test.wikipedia.org, 2022-11-08T23:00:11.292Z,...",eas.ve.loaded,"(Test5555 Name, None, 54295, None, 0, None, 0,...","(None, 9, None, None, 47306, None, True, None,...","{'os_family': 'Mac OS X', 'browser_major': '10...",True,"(wikipedia, test, [], org, wikipedia)",codfw,2022,11,8,23
59,None,"(None, mediawiki_js, desktop_browser)","{'abort_mechanism': ('string', 'navigate'), 'i...",2022-11-08T23:00:09.844Z,"(None, None, None, {'user-agent': 'Mozilla/5.0...","(None, None, None, None, None, 1.40.0-wmf.8, N...","(test.wikipedia.org, 2022-11-08T23:00:11.302Z,...",eas.ve.abort,"(Test5555 Name, None, 54295, None, 0, None, 0,...","(None, 9, None, None, 47306, None, True, None,...","{'os_family': 'Mac OS X', 'browser_major': '10...",True,"(wikipedia, test, [], org, wikipedia)",codfw,2022,11,8,23


These events made it from Browser to the table within 4 hours! 

In [18]:
query2=spark.run('''
SELECT 
  name, 
  count(1) AS events
  FROM event.mediawiki_edit_attempt
  WHERE performer.id=47306
  GROUP BY name 
  ORDER BY events desc
''')

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


The table has VisualEditor and Discussiontools events. None of the wikitext events are in the table, as they didnt get fired when I checked on the browser Console

In [19]:
query2

,name,events
0,eas.dt.loaded,9
1,eas.dt.init,9
2,eas.dt.ready,9
3,eas.dt.first_change,8
4,eas.dt.save_attempt,6
5,eas.dt.save_success,6
6,eas.dt.save_intent,6
7,eas.dt.abort,3
8,eas.ve.init,1
9,eas.ve.loaded,1


In [20]:
query3=spark.run('''
SELECT 
  *
  FROM event.mediawiki_edit_attempt
  WHERE performer.id=47306
  AND name like 'eas.dt%'
  ORDER BY dt 
''')

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


Verified that all Discussion Tools events are in the table and with correct timestamps

In [33]:
query3.tail()

,_schema,agent,custom_data,dt,http,mediawiki,meta,name,page,performer,user_agent_map,is_wmf_domain,normalized_host,datacenter,year,month,day,hour
51,None,"(None, mediawiki_js, desktop_browser)","{'integration': ('string', 'discussiontools'),...",2022-11-08T22:54:33.684Z,"(None, None, None, {'user-agent': 'Mozilla/5.0...","(None, None, None, None, None, 1.40.0-wmf.8, N...","(test.wikipedia.org, 2022-11-08T22:55:03.733Z,...",eas.dt.init,"(Test5555 Name, None, 147141, None, 1, None, 5...","(None, 5, None, None, 47306, None, True, None,...","{'os_family': 'Mac OS X', 'browser_major': '10...",True,"(wikipedia, test, [], org, wikipedia)",codfw,2022,11,8,22
52,None,"(None, mediawiki_js, desktop_browser)","{'integration': ('string', 'discussiontools'),...",2022-11-08T22:54:33.876Z,"(None, None, None, {'user-agent': 'Mozilla/5.0...","(None, None, None, None, None, 1.40.0-wmf.8, N...","(test.wikipedia.org, 2022-11-08T22:55:03.734Z,...",eas.dt.loaded,"(Test5555 Name, None, 147141, None, 1, None, 5...","(None, 5, None, None, 47306, None, True, None,...","{'os_family': 'Mac OS X', 'browser_major': '10...",True,"(wikipedia, test, [], org, wikipedia)",codfw,2022,11,8,22
53,None,"(None, mediawiki_js, desktop_browser)","{'integration': ('string', 'discussiontools'),...",2022-11-08T22:54:33.876Z,"(None, None, None, {'user-agent': 'Mozilla/5.0...","(None, None, None, None, None, 1.40.0-wmf.8, N...","(test.wikipedia.org, 2022-11-08T22:55:03.734Z,...",eas.dt.ready,"(Test5555 Name, None, 147141, None, 1, None, 5...","(None, 5, None, None, 47306, None, True, None,...","{'os_family': 'Mac OS X', 'browser_major': '10...",True,"(wikipedia, test, [], org, wikipedia)",codfw,2022,11,8,22
54,None,"(None, mediawiki_js, desktop_browser)","{'integration': ('string', 'discussiontools'),...",2022-11-08T22:54:35.255Z,"(None, None, None, {'user-agent': 'Mozilla/5.0...","(None, None, None, None, None, 1.40.0-wmf.8, N...","(test.wikipedia.org, 2022-11-08T22:55:03.734Z,...",eas.dt.first_change,"(Test5555 Name, None, 147141, None, 1, None, 5...","(None, 5, None, None, 47306, None, True, None,...","{'os_family': 'Mac OS X', 'browser_major': '10...",True,"(wikipedia, test, [], org, wikipedia)",codfw,2022,11,8,22
55,None,"(None, mediawiki_js, desktop_browser)","{'integration': ('string', 'discussiontools'),...",2022-11-08T22:55:55.550Z,"(None, None, None, {'user-agent': 'Mozilla/5.0...","(None, None, None, None, None, 1.40.0-wmf.8, N...","(test.wikipedia.org, 2022-11-08T22:56:00.991Z,...",eas.dt.abort,"(Test5555 Name, None, 147141, None, 1, None, 5...","(None, 5, None, None, 47306, None, True, None,...","{'os_family': 'Mac OS X', 'browser_major': '10...",True,"(wikipedia, test, [], org, wikipedia)",codfw,2022,11,8,22


Visual Editor events have init, ready, loaded and abort events loaded but I dont see save Intent, save Attempt and save Success events in the table, nor did I see it fired in the Browser Console 

### Round 2 
#### Date: Nov 9, 2022
User: Mayakp.wiki  
User_id: 47173  


In [26]:
query4=spark.run('''
SELECT 
  name, 
  count(1) AS events
  FROM event.mediawiki_edit_attempt
  WHERE performer.id=47173
  GROUP BY name 
  ORDER BY events desc
''')

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
22/11/09 21:30:10 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
22/11/09 21:30:10 WARN Utils: Service 'sparkDriver' could not bind on port 12000. Attempting port 12001.
22/11/09 21:30:10 WARN Utils: Service 'sparkDriver' could not bind on port 12001. Attempting port 12002.
22/11/09 21:30:10 WARN Utils: Service 'sparkDriver' could not bind on port 12002. Attempting port 12003.
22/11/09 21:30:10 WARN Utils: Service 'sparkDriver' could not bind on port 12003. Attempting port 12004.
22/11/09 21:30:10 WARN Utils: Service 'sparkDriver' could not bind on port 12004. Attempting port 12005.
22/11/09 21:30:10 WARN Utils: Service 'sparkDriver' could not bind on port 12005. Attempting port 12006.
22/11/09 21:30:10 WARN Utils: Service 'sparkDriver' could not bind on port 12006. Attempting port 12007.
22/11/

In [34]:
query4.sort_values(by=['events'],ascending=False)

,name,events
0,eas.ve.first_change,2
1,eas.ve.loaded,2
2,eas.ve.ready,2
3,eas.ve.init,2
4,eas.ve.save_intent,1
5,eas.ve.save_attempt,1
6,eas.ve.abort,1
7,eas.ve.save_success,1


In [30]:
query5=spark.run('''
SELECT 
  *
  FROM event.mediawiki_edit_attempt
  WHERE performer.id=47173
  ORDER BY dt
''')

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
22/11/09 23:19:43 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
22/11/09 23:19:43 WARN Utils: Service 'sparkDriver' could not bind on port 12000. Attempting port 12001.
22/11/09 23:19:43 WARN Utils: Service 'sparkDriver' could not bind on port 12001. Attempting port 12002.
22/11/09 23:19:43 WARN Utils: Service 'sparkDriver' could not bind on port 12002. Attempting port 12003.
22/11/09 23:19:43 WARN Utils: Service 'sparkDriver' could not bind on port 12003. Attempting port 12004.
22/11/09 23:19:43 WARN Utils: Service 'sparkDriver' could not bind on port 12004. Attempting port 12005.
22/11/09 23:19:43 WARN Utils: Service 'sparkDriver' could not bind on port 12005. Attempting port 12006.
22/11/09 23:19:43 WARN Utils: Service 'sparkDriver' could not bind on port 12006. Attempting port 12007.
22/11/

In [31]:
query5

,_schema,agent,custom_data,dt,http,mediawiki,meta,name,page,performer,user_agent_map,is_wmf_domain,normalized_host,datacenter,year,month,day,hour
0,None,"(None, mediawiki_js, desktop_browser)","{'integration': ('string', 'page'), 'editing_s...",2022-11-09T18:20:09.545Z,"(None, None, None, {'user-agent': 'Mozilla/5.0...","(False, None, None, None, None, 1.40.0-wmf.8, ...","(test.wikipedia.org, 2022-11-09T18:20:16.926Z,...",eas.ve.init,"(Page376, None, 49447, None, 0, None, 0, None,...","(None, 1, None, None, 47173, None, True, None,...","{'os_family': 'Mac OS X', 'browser_major': '98...",True,"(wikipedia, test, [], org, wikipedia)",codfw,2022,11,9,18
1,None,"(None, mediawiki_js, desktop_browser)","{'integration': ('string', 'page'), 'editing_s...",2022-11-09T18:20:10.317Z,"(None, None, None, {'user-agent': 'Mozilla/5.0...","(False, None, None, None, None, 1.40.0-wmf.8, ...","(test.wikipedia.org, 2022-11-09T18:20:16.929Z,...",eas.ve.ready,"(Page376, None, 49447, None, 0, None, 0, None,...","(None, 1, None, None, 47173, None, True, None,...","{'os_family': 'Mac OS X', 'browser_major': '98...",True,"(wikipedia, test, [], org, wikipedia)",codfw,2022,11,9,18
2,None,"(None, mediawiki_js, desktop_browser)","{'integration': ('string', 'page'), 'editing_s...",2022-11-09T18:20:10.318Z,"(None, None, None, {'user-agent': 'Mozilla/5.0...","(False, None, None, None, None, 1.40.0-wmf.8, ...","(test.wikipedia.org, 2022-11-09T18:20:16.929Z,...",eas.ve.loaded,"(Page376, None, 49447, None, 0, None, 0, None,...","(None, 1, None, None, 47173, None, True, None,...","{'os_family': 'Mac OS X', 'browser_major': '98...",True,"(wikipedia, test, [], org, wikipedia)",codfw,2022,11,9,18
3,None,"(None, mediawiki_js, desktop_browser)","{'integration': ('string', 'page'), 'editing_s...",2022-11-09T18:21:02.254Z,"(None, None, None, {'user-agent': 'Mozilla/5.0...","(False, None, None, None, None, 1.40.0-wmf.8, ...","(test.wikipedia.org, 2022-11-09T18:21:09.927Z,...",eas.ve.first_change,"(Page376, None, 49447, None, 0, None, 0, None,...","(None, 1, None, None, 47173, None, True, None,...","{'os_family': 'Mac OS X', 'browser_major': '98...",True,"(wikipedia, test, [], org, wikipedia)",codfw,2022,11,9,18
4,None,"(None, mediawiki_js, desktop_browser)","{'integration': ('string', 'page'), 'save_inte...",2022-11-09T18:22:52.937Z,"(None, None, None, {'user-agent': 'Mozilla/5.0...","(False, None, None, None, None, 1.40.0-wmf.8, ...","(test.wikipedia.org, 2022-11-09T18:23:18.739Z,...",eas.ve.save_intent,"(Page376, None, 49447, None, 0, None, 0, None,...","(None, 1, None, None, 47173, None, True, None,...","{'os_family': 'Mac OS X', 'browser_major': '98...",True,"(wikipedia, test, [], org, wikipedia)",codfw,2022,11,9,18
5,None,"(None, mediawiki_js, desktop_browser)","{'integration': ('string', 'page'), 'editing_s...",2022-11-09T18:23:16.793Z,"(None, None, None, {'user-agent': 'Mozilla/5.0...","(False, None, None, None, None, 1.40.0-wmf.8, ...","(test.wikipedia.org, 2022-11-09T18:23:18.740Z,...",eas.ve.save_attempt,"(Page376, None, 49447, None, 0, None, 0, None,...","(None, 1, None, None, 47173, None, True, None,...","{'os_family': 'Mac OS X', 'browser_major': '98...",True,"(wikipedia, test, [], org, wikipedia)",codfw,2022,11,9,18
6,None,"(None, mediawiki_js, desktop_browser)","{'integration': ('string', 'page'), 'editing_s...",2022-11-09T18:23:17.725Z,"(None, None, None, {'user-agent': 'Mozilla/5.0...","(False, None, None, None, None, 1.40.0-wmf.8, ...","(test.wikipedia.org, 2022-11-09T18:23:18.741Z,...",eas.ve.save_success,"(Page376, None, 49447, None, 0, None, 0, None,...","(None, 1, None, None, 47173, None, True, None,...","{'os_family': 'Mac OS X', 'browser_major': '98...",True,"(wikipedia, test, [], org, wikipedia)",codfw,2022,11,9,18
7,None,"(None, mediawiki_js, desktop_browser)","{'integration': ('string', 'page'), 'editing_s...",2022-11-09T18:27:42.264Z,"(None, None, None, {'user-agent': 'Mozilla/5.0...","(False, None, None, None, None, 1.40.0-wmf.8, ...","(test.w

This time around all Visual Editor events were logged successfully. 